In [1]:
!git clone https://github.com/GuilhermeMarcon/graph-committee-e2e-absa.git

Cloning into 'graph-committee-e2e-absa'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 5), reused 3 (delta 1), pack-reused 0
Receiving objects: 100% (27/27), 712.10 KiB | 3.10 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.4 MB/s eta 0:00:00


# Creating the embeddings

In [3]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model/tokenizer
model_name = 'bert-base-multilingual-cased'
# model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Preprocess text
text = "This is an example sentence."
tokens = tokenizer.encode(text, add_special_tokens=True)

# Convert token IDs to PyTorch tensor
tokens_tensor = torch.tensor([tokens])

# Generate embeddings
with torch.no_grad():
    outputs = model(tokens_tensor)
    embedding = outputs.pooler_output.squeeze(0)
embedding

tensor([ 0.0118, -0.3553,  0.4665, -0.2460, -0.1493,  0.2987,  0.3957,  0.3839,
        -0.3758,  0.3777, -0.1596, -0.3166, -0.3941, -0.2168,  0.3015, -0.1415,
         0.7290,  0.2471,  0.2568, -0.1940, -0.9998, -0.3132, -0.1743, -0.3881,
        -0.4211,  0.3379, -0.3137,  0.5097,  0.1776, -0.4438,  0.3100, -0.9998,
         0.7531,  0.5151,  0.3102, -0.2703,  0.0209,  0.2605,  0.2450, -0.4463,
        -0.3100,  0.0297, -0.3389,  0.1329, -0.1532, -0.5220, -0.2656,  0.1486,
        -0.4792,  0.1321, -0.0032,  0.2347,  0.5217,  0.2278,  0.3569, -0.0136,
         0.2426,  0.2342,  0.4652, -0.4817, -0.0228,  0.5353,  0.4132, -0.2530,
        -0.1787, -0.1968,  0.3854, -0.0431,  0.5215, -0.1971, -0.3556, -0.3046,
        -0.3084,  0.0566,  0.2902, -0.3673,  0.4513,  0.2999,  0.2436, -0.1398,
        -0.4545, -0.4901, -0.4418,  0.3406, -0.3021,  0.5242,  0.4530, -0.4489,
         0.3938, -0.2027,  0.2232,  0.4214, -0.3491,  0.4418, -0.2454, -0.3009,
        -0.7882, -0.4126, -0.2740, -0.35

In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval

def read_df(filename, clean_empty=True):
    print("Reading", filename)
    df = pd.read_excel(filename)
    if(clean_empty): print("Original size", len(df))
    for col in df.columns[1:]:
        df[col] = df[col].apply(literal_eval)
        if(clean_empty): df = df[df[col].apply(len) != 0]
    print("Final size", len(df))
    return df

In [4]:
ROOT = '/content/graph-committee-e2e-absa/data/'
DATASET = 'hotel22'

In [5]:
test = read_df(ROOT+DATASET+'/test.xlsx', clean_empty=False)
display(test.head())

Reading /content/graph-committee-e2e-absa/data/hotel22/test.xlsx
Final size 84


,review,annotation,hotel22-bert-base-multilingual-cased-e2eabsa-linear,hotel22-bert-base-multilingual-cased-e2eabsa-san,hotel22-bert-base-multilingual-cased-e2eabsa-tfm,hotel22-bert-base-multilingual-cased-e2eabsa-crf,hotel22-bert-base-multilingual-cased-e2eabsa-gru,committee-voting,rest14-bert-base-multilingual-cased-e2eabsa-linear,rest14-bert-base-multilingual-cased-e2eabsa-san,...,multidomain-bert-base-multilingual-cased-e2eabsa-san,multidomain-bert-base-multilingual-cased-e2eabsa-tfm,multidomain-bert-base-multilingual-cased-e2eabsa-crf,multidomain-bert-base-multilingual-cased-e2eabsa-gru,gpt3.5,gpt3.5-bert-base-multilingual-cased-e2eabsa-linear,gpt3.5-bert-base-multilingual-cased-e2eabsa-san,gpt3.5-bert-base-multilingual-cased-e2eabsa-tfm,gpt3.5-bert-base-multilingual-cased-e2eabsa-crf,gpt3.5-bert-base-multilingual-cased-e2eabsa-gru
0,Em abril de 2012 nos hospedamos no hotel refer...,"[(lojas, POS), (hotel, POS), (quartos, POS), (...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (lojas, POS), (Funcionari...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur...",...,"[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (lojas, ...","[(localizacao, POS), (metro, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (Funcion...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[(rua, NEU), (café da manhã, POS), (localizaçã...","[(hotel, POS), (localização, POS), (antendimen...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (antendimen...","[(localização, POS), (louvre, POS), (metro, PO...","[(localização, POS), (antendimento, POS), (caf...",...,"[(localização, POS), (louvre, POS), (antendime...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (louvre, POS), (moulan, P...","[(localização, POS), (antendimento, POS), (caf...","[(hotel, POS), (localização, POS), (atendiment...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (louvre, PO...","[(hotel, POS), (localização, POS), (caminhar, ...","[(hotel, POS), (localização, POS), (moulan rou...","[(hotel, POS), (localização, POS), (louvre, PO..."
2,"Localização excelente, atendimento e serviços ...","[(cama, NEU), (serviços, POS), (Café da manhã,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...",...,"[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (quarto, POS), (banheiro,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv..."
3,Muito satisfe

In [6]:
from tqdm import tqdm
tqdm.pandas()

def text_to_embedding(text):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    tokens_tensor = torch.tensor([tokens])
    with torch.no_grad():
        outputs = model(tokens_tensor)
        embedding = outputs.pooler_output.squeeze(0).cpu().detach().numpy()
    return list(embedding)

def entities_to_embeddings(entities):
    return [text_to_embedding(x[0]) for x in entities]

def create_df_embeddings(df, text_col='review'):
    print("Embedding sentences...")
    original_cols = list(df.columns)
    # print(original_cols)
    df['text_emb'] = df[text_col].progress_apply(text_to_embedding)
    for col in original_cols[1:]:
        if col.endswith('_emb'): continue
        print('Embedding entities "'+str(col)+'" ...')
        df[col+'_ents_emb'] = df[col].progress_apply(entities_to_embeddings)

create_df_embeddings(test, text_col='review')

Embedding sentences...


100%|██████████| 84/84 [00:22<00:00,  3.68it/s]


Embedding entities "annotation" ...


100%|██████████| 84/84 [00:21<00:00,  3.97it/s]


Embedding entities "hotel22-bert-base-multilingual-cased-e2eabsa-linear" ...


...

100%|██████████| 84/84 [00:51<00:00,  1.63it/s]


Embedding entities "gpt3.5-bert-base-multilingual-cased-e2eabsa-crf" ...


100%|██████████| 84/84 [00:51<00:00,  1.64it/s]


Embedding entities "gpt3.5-bert-base-multilingual-cased-e2eabsa-gru" ...


100%|██████████| 84/84 [00:49<00:00,  1.69it/s]


In [7]:
test.to_pickle(ROOT+DATASET+'/test_embs.pkl')

In [17]:
import pandas as pd

test = pd.read_pickle(ROOT+DATASET+'/test_embs.pkl').rename(columns={'review':'text', 'snippet':'text'})
test.head()

,text,annotation,hotel22-bert-base-multilingual-cased-e2eabsa-linear,hotel22-bert-base-multilingual-cased-e2eabsa-san,hotel22-bert-base-multilingual-cased-e2eabsa-tfm,hotel22-bert-base-multilingual-cased-e2eabsa-crf,hotel22-bert-base-multilingual-cased-e2eabsa-gru,committee-voting,rest14-bert-base-multilingual-cased-e2eabsa-linear,rest14-bert-base-multilingual-cased-e2eabsa-san,...,multidomain-bert-base-multilingual-cased-e2eabsa-san_ents_emb,multidomain-bert-base-multilingual-cased-e2eabsa-tfm_ents_emb,multidomain-bert-base-multilingual-cased-e2eabsa-crf_ents_emb,multidomain-bert-base-multilingual-cased-e2eabsa-gru_ents_emb,gpt3.5_ents_emb,gpt3.5-bert-base-multilingual-cased-e2eabsa-linear_ents_emb,gpt3.5-bert-base-multilingual-cased-e2eabsa-san_ents_emb,gpt3.5-bert-base-multilingual-cased-e2eabsa-tfm_ents_emb,gpt3.5-bert-base-multilingual-cased-e2eabsa-crf_ents_emb,gpt3.5-bert-base-multilingual-cased-e2eabsa-gru_ents_emb
0,Em abril de 2012 nos hospedamos no hotel refer...,"[(lojas, POS), (hotel, POS), (quartos, POS), (...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (lojas, POS), (Funcionari...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(hotel, POS), (localizacao, POS), (farmacia, ...","[(localizacao, POS), (farmacia, POS), (restaur...","[(localizacao, POS), (farmacia, POS), (restaur...",...,"[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.14371592, -0.0070755435, 0.1792188, -0.040...","[[0.14371592, -0.0070755435, 0.1792188, -0.040..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[(rua, NEU), (café da manhã, POS), (localizaçã...","[(hotel, POS), (localização, POS), (antendimen...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (hotel, POS...","[(hotel, POS), (localização, POS), (antendimen...","[(localização, POS), (louvre, POS), (metro, PO...","[(localização, POS), (antendimento, POS), (caf...",...,"[[0.4806645, -0.21390055, 0.03182074, -0.32436...","[[0.4806645, -0.21390055, 0.03182074, -0.32436...","[[0.4806645, -0.21390055, 0.03182074, -0.32436...","[[0.4806645, -0.21390055, 0.03182074, -0.32436...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897...","[[0.17371255, -0.14632678, 0.27990723, -0.1897..."
2,"Localização excelente, atendimento e serviços ...","[(cama, NEU), (serviços, POS), (Café da manhã,...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...","[(Localização, POS), (atendimento, POS), (serv...",...,"[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3653136, -0.14689...","[[0.24602753, -0.18310724, 0.3

# Graph Regularization

In [18]:
import networkx as nx
import numpy as np
from scipy.spatial import distance
from tqdm import tqdm

def create_graph_from_df(df, text='text', ignore_model=None, columns=2, mi=0.9, distance_threshold=0.25):
    G = nx.Graph()

    for col in df.columns[columns:]:
        if col.endswith('_emb'): continue
        if ignore_model is not None:
            if type(ignore_model) is not list:
                ignore_model = [ignore_model]
            to_skip = False
            for igm in ignore_model:
                if igm in col:
                    print("Jumping", col)
                    to_skip = True
                    break
            if to_skip: continue

        # if not col.startswith('hotel22'): continue
        print("Adding", col, "to graph.")

        for i, row in df.iterrows():
            # display(row[col])
            annotation = np.asarray(df.iloc[i]['annotation'])
            if len(annotation) == 0: continue
            # print(annotation)
            annotation_aspects = annotation[:, 0]
            for j, (aspect, sentiment) in enumerate(row[col]):
                node_id = f"{i}_{col}_{aspect}_{sentiment}"
                # print(node_id)
                aspect_embedding = mi * np.array(row[col+'_ents_emb'][j]) + (1.0-mi) * np.array(row[text+'_emb'])

                annotation_i = np.where(annotation_aspects == aspect)[0]
                # print(annotation_i)
                if len(annotation_i) >= 1:
                    annotation_sentiment = annotation[annotation_i, 1][0]
                else:
                    annotation_sentiment = 'NONE'
                    continue

                node_data = {
                    'text': row[text],
                    'model': col,
                    'aspect': aspect,
                    'sentiment': sentiment,
                    'annotation': annotation_sentiment,
                    'embedding': aspect_embedding
                }
                # print(node_id, sentiment, annotation_sentiment)
                G.add_node(node_id, **node_data)
            #     break
        # break

    for node1, data1 in tqdm(G.nodes(data=True)):
        for node2, data2 in G.nodes(data=True):
            # Linking same aspects with different sentiments
            if node1 != node2 and data1['aspect'].lower() == data2['aspect'].lower() and data1['sentiment'] != data2['sentiment']:
                weight = sum([a*b for a,b in zip(data1['embedding'], data2['embedding'])])
                G.add_edge(node1, node2, weight=weight)
            # Linking close embeddings with same sentiments
            elif node1 != node2 and data1['sentiment'] == data2['sentiment']:
                # dist = distance.cosine(data1['embedding'], data2['embedding'])
                dist = 1.0 - np.dot(data1['embedding'], data2['embedding']) / (np.linalg.norm(data1['embedding']) * np.linalg.norm(data2['embedding']))
                if dist < distance_threshold:
                    G.add_edge(node1, node2, weight=dist)
    return G

In [19]:
G = create_graph_from_df(test, mi=0.9, distance_threshold=0.1, ignore_model=['rest14', 'laptop14', 'graph', 'multidomain', 'committee', 'gpt3.5'])

Adding hotel22-bert-base-multilingual-cased-e2eabsa-linear to graph.
Adding hotel22-bert-base-multilingual-cased-e2eabsa-san to graph.
Adding hotel22-bert-base-multilingual-cased-e2eabsa-tfm to graph.
Adding hotel22-bert-base-multilingual-cased-e2eabsa-crf to graph.
Adding hotel22-bert-base-multilingual-cased-e2eabsa-gru to graph.
Jumping committee-voting
Jumping rest14-bert-base-multilingual-cased-e2eabsa-linear
Jumping rest14-bert-base-multilingual-cased-e2eabsa-san
Jumping rest14-bert-base-multilingual-cased-e2eabsa-tfm
Jumping rest14-bert-base-multilingual-cased-e2eabsa-crf
Jumping rest14-bert-base-multilingual-cased-e2eabsa-gru
Jumping graph-bert-base-multilingual-cased-e2eabsa-linear
Jumping graph-bert-base-multilingual-cased-e2eabsa-san
Jumping graph-bert-base-multilingual-cased-e2eabsa-tfm
Jumping graph-bert-base-multilingual-cased-e2eabsa-crf
Jumping graph-bert-base-multilingual-cased-e2eabsa-gru
Jumping committee-graph
Jumping multidomain-bert-base-multilingual-cased-e2eabsa-

100%|██████████| 1396/1396 [00:37<00:00, 37.26it/s]


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
import numpy as np
from tqdm.notebook import tqdm
import random

def regularization(G, num_labels=3, iterations=10, mi=0.1, save_plots=False, plot_max_depth=4, plot_max_nodes=20, plot_skip=0, plot_title='Graph Regularization'):

    nodes = []

    # inicializando vetor f para todos os nodes
    for node in G.nodes():
        if 'f' not in G.nodes[node]:
            G.nodes[node]['f'] = np.array([0.0]*num_labels)
            G.nodes[node]['y'] = np.array([0.0]*num_labels)
            if G.nodes[node]['sentiment']   == 'NEG':
                G.nodes[node]['y'][0] = 1.0
                G.nodes[node]['f'][0] = 1.0
            elif G.nodes[node]['sentiment'] == 'NEU':
                G.nodes[node]['y'][1] = 1.0
                G.nodes[node]['f'][1] = 1.0
            elif G.nodes[node]['sentiment'] == 'POS':
                G.nodes[node]['y'][2] = 1.0
                G.nodes[node]['f'][2] = 1.0
        nodes.append(node)

    pbar = tqdm(range(0, iterations))

    for iteration in pbar:
        random.shuffle(nodes)
        energy = 0.0

        for node in nodes:
            f_new = np.array([0.0]*num_labels)
            f_old = np.array(G.nodes[node]['f'])*1.0
            sum_w = 0.0

            for neighbor in G.neighbors(node):

                  w = 0.0
                  if 'weight' in G[node][neighbor]:
                      w = G[node][neighbor]['weight']

                  f_new += w*G.nodes[neighbor]['f']

                  sum_w += w

            if sum_w!=0.0:
              f_new /= sum_w
            else:
              f_new = f_old

            G.nodes[node]['f'] = f_new*1.0

            G.nodes[node]['f'] = G.nodes[node]['y'] * mi + G.nodes[node]['f']*(1.0-mi)

            energy += np.linalg.norm(f_new-f_old)

        iteration += 1

        message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
        # print(message)
        pbar.set_description(message)

    return G

def reset_regularization(G):
    for node in G.nodes():
        if 'f' not in G.nodes[node]: return
        break

    for node in G.nodes():
        G.nodes[node]['f'] = G.nodes[node]['y']
# reset_regularization(G)

In [22]:
regularization(G, mi=0.5, iterations=10)

  0%|          | 0/10 [00:00<?, ?it/s]

# Graph Committee

In [24]:
import numpy as np
import pandas as pd

isent_to_sent = {0:'NEG', 1:'NEU', 2:'POS'}

# graph, base dataframe
def graph_committee_to_data(G, bdf):
    df = bdf[['text', 'annotation']].copy()

    id_aspect_dict = [{} for _ in range(len(df))]

    for node in G.nodes():
        id, model, aspect, sentiment = node.split('_')
        id = int(id)

        if aspect not in id_aspect_dict[id]:
            id_aspect_dict[id][aspect] = []
        id_aspect_dict[id][aspect].append(G.nodes[node]['f'])

    data = []
    for id in range(len(df)):
        data.append([])
        for aspect in id_aspect_dict[id]:
            new_sent = isent_to_sent[np.argmax(np.mean(id_aspect_dict[id][aspect], axis=0))]
            data[-1].append((aspect, new_sent))
    return data

gdf = graph_committee_to_data(G, test)
gdf

[[('hotel', 'POS'),
  ('limpeza', 'POS'),
  ('quartos', 'POS'),
  ('custo-benefício', 'POS'),
  ('lojas', 'POS')],
 [('hotel', 'POS'),
  ('localização', 'POS'),
  ('café da manhã', 'POS'),
  ('rua', 'POS')],
 [('Localização', 'POS'),
  ('atendimento', 'POS'),
  ('serviços', 'POS'),
  ('hotel', 'POS'),
  ('cama', 'POS'),
  ('Café da manhã', 'POS')],
 [('localização', 'POS'), ('piscinas', 'NEU')],
 [('serviço', 'POS'), ('piscina', 'POS'), ('quarto', 'POS'), ('cama', 'POS')],
 [('Hotel', 'POS'), ('quarto', 'POS'), ('internet', 'POS')],
 [('quarto', 'POS'), ('preço', 'POS'), ('chuveiro', 'POS')],
 [('hotel', 'POS'),
  ('localização', 'NEG'),
  ('elevador', 'NEG'),
  ('estacionamento', 'NEG'),
  ('quartos', 'NEG')],
 [('quarto', 'POS'),
  ('atendimento', 'POS'),
  ('hotel', 'POS'),
  ('internet', 'NEG')],
 [('Hotel', 'POS'), ('quarto', 'POS'), ('chuveiro', 'POS')],
 [('hotel', 'POS'),
  ('quartos', 'POS'),
  ('café da manhã', 'POS'),
  ('funcionários', 'POS'),
  ('localização', 'POS'),
  ('

In [25]:
test = pd.read_excel(ROOT+DATASET+'/test.xlsx')
test['committee-graph'] = gdf
test.to_excel(ROOT+DATASET+'/test.xlsx', index=False)
test

,review,annotation,hotel22-bert-base-multilingual-cased-e2eabsa-linear,hotel22-bert-base-multilingual-cased-e2eabsa-san,hotel22-bert-base-multilingual-cased-e2eabsa-tfm,hotel22-bert-base-multilingual-cased-e2eabsa-crf,hotel22-bert-base-multilingual-cased-e2eabsa-gru,committee-voting,rest14-bert-base-multilingual-cased-e2eabsa-linear,rest14-bert-base-multilingual-cased-e2eabsa-san,...,multidomain-bert-base-multilingual-cased-e2eabsa-san,multidomain-bert-base-multilingual-cased-e2eabsa-tfm,multidomain-bert-base-multilingual-cased-e2eabsa-crf,multidomain-bert-base-multilingual-cased-e2eabsa-gru,gpt3.5,gpt3.5-bert-base-multilingual-cased-e2eabsa-linear,gpt3.5-bert-base-multilingual-cased-e2eabsa-san,gpt3.5-bert-base-multilingual-cased-e2eabsa-tfm,gpt3.5-bert-base-multilingual-cased-e2eabsa-crf,gpt3.5-bert-base-multilingual-cased-e2eabsa-gru
0,Em abril de 2012 nos hospedamos no hotel refer...,"[('lojas', 'POS'), ('hotel', 'POS'), ('quartos...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('localizacao', 'POS'), ('lojas', 'POS'), ('F...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...",...,"[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('metro', 'POS'), ('f...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('hotel', 'POS'), ('localizacao', 'POS'), ('f...","[('localizacao', 'POS'), ('farmacia', 'POS'), ...","[('localizacao', 'POS'), ('farmacia', 'POS'), ..."
1,"EU fiquei uma semana neste hotel, é maravilhos...","[('rua', 'NEU'), ('café da manhã', 'POS'), ('l...","[('hotel', 'POS'), ('localização', 'POS'), ('a...","[('hotel', 'POS'), ('localização', 'POS'), ('h...","[('hotel', 'POS'), ('localização', 'POS'), ('h...","[('hotel', 'POS'), ('localização', 'POS'), ('h...","[('hotel', 'POS'), ('localização', 'POS'), ('h...","[('hotel', 'POS'), ('localização', 'POS'), ('a...","[('localização', 'POS'), ('louvre', 'POS'), ('...","[('localização', 'POS'), ('antendimento', 'POS...",...,"[('localização', 'POS'), ('louvre', 'POS'), ('...","[('localização', 'POS'), ('louvre', 'POS'), ('...","[('localização', 'POS'), ('louvre', 'POS'), ('...","[('localização', 'POS'), ('antendimento', 'POS...","[('hotel', 'POS'), ('localização', 'POS'), ('a...","[('hotel', 'POS'), ('localização', 'POS'), ('l...","[('hotel', 'POS'), ('localização', 'POS'), ('l...","[('hotel', 'POS'), ('localização', 'POS'), ('c...","[('hotel', 'POS'), ('localização', 'POS'), ('m...","[('hotel', 'POS'), ('localização', 'POS'), ('l..."
2,"Localização excelente, atendimento e serviços ...","[('cama', 'NEU'), ('serviços', 'POS'), ('Café ...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...",...,"[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('quarto', 'POS'), ('...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'...","[('Localização', 'POS'), ('atendimento', 'POS'..."
3,Muito satisfe